# Assignment 7

### Part A

In [3]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [4]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [5]:
def expected_return(P, rf, mkt_rp):
    # Calculating the expected return of each security
    P['Expected Return'] = rf + P['Beta'] * mkt_rp
    # Calculating the weighted average of the expected returns
    total_investment = P['Amount Invested'].sum()
    P['Weight'] = P['Amount Invested'] / total_investment
    portfolio_return = (P['Expected Return'] * P['Weight']).sum()
    return portfolio_return

In [6]:
expected_return(portfolio, 0.02, 0.05)

0.086

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [7]:
def volatility(P, cor):
    # Extracting weights and volatilities
    weights = P['Amount Invested'] / P['Amount Invested'].sum()
    volatilities = P['Volatility']
    
    # Calculating portfolio variance
    portfolio_variance = 0
    n = len(weights)
    for i in range(n):
        for j in range(n):
            if i == j:  # Variance term
                portfolio_variance += (weights[i] ** 2) * (volatilities[i] ** 2)
            else:  # Covariance term
                portfolio_variance += weights[i] * weights[j] * volatilities[i] * volatilities[j] * cor
    
    # Taking square root to get volatility
    portfolio_volatility = np.sqrt(portfolio_variance)
    return portfolio_volatility

In [8]:
volatility(portfolio, -0.2)

0.24083189157584592

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [9]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    return rf + (vol / mkt_vol) * mkt_rp


In [10]:
expected_return2(0.02, 0.05, 0.16, 0.2408)

0.09525

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [11]:
def CAPM_beta(return_path, factors_path):
    # Read the CSV files into DataFrames
    returns_df = pd.read_csv(return_path)
    factors_df = pd.read_csv(factors_path)

    # Convert the 'Mkt-RF' and 'RF' in the factors DataFrame from percentages to decimals
    factors_df['Mkt-RF'] /= 100
    factors_df['RF'] /= 100

    # Convert 'date' column to string and create 'year' and 'month' columns for merging
    factors_df['date'] = factors_df['date'].astype(str)
    factors_df['year'] = factors_df['date'].str.slice(0, 4).astype(int)
    factors_df['month'] = factors_df['date'].str.slice(4, 6).astype(int)

    # Merge the returns and factors DataFrames on 'year' and 'month'
    merged_df = pd.merge(returns_df, factors_df, on=['year', 'month'])

    # Calculate the excess returns for each security
    merged_df['Excess_RET'] = merged_df['RET'] - merged_df['RF']

    # Calculate CAPM beta for each CUSIP
    beta_values = merged_df.groupby('CUSIP').apply(
        lambda df: np.cov(df['Excess_RET'], df['Mkt-RF'])[0][1] / np.var(df['Mkt-RF'])
    )

    # Create the output DataFrame with CUSIP, expected return, and CAPM-beta
    output_df = beta_values.reset_index().rename(columns={0: 'BETA'})
    output_df['EXP_RETURN'] = None  # Placeholder for the expected return

    return output_df

In [12]:
CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.CSV")

CUSIP      BETA EXP_RETURN
0     00081T10  2.603590       None
1     00108M10  0.742466       None
2     00130H10  1.460965       None
3     00163U10  0.977903       None
4     00182C10  1.412225       None
...        ...       ...        ...
3220   9959910  0.765642       None
3221   9972410  1.284197       None
3222   9972510  1.070395       None
3223   9984910  1.884472       None
3224   9985510  1.171598       None

[3225 rows x 3 columns]

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [13]:
def slope(data):
    
    return 0

In [14]:
capm_beta_df = CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.CSV")
slope(capm_beta_df)

0